In [1]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd
import re

In [2]:
df = pd.read_csv("PPR-ALL.csv", encoding='unicode_escape')

/var/folders/55/j0v2zlbn1575rkmztsycxlfw0000gn/T/ipykernel_5914/96290156.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("PPR-ALL.csv", encoding='unicode_escape')


In [3]:
df.rename(columns = {"Date of Sale (dd/mm/yyyy)":"Date"},inplace=True)

In [4]:
df['Date'] = pd.to_datetime(df['Date'], infer_datetime_format=True, format='mixed')

/var/folders/55/j0v2zlbn1575rkmztsycxlfw0000gn/T/ipykernel_5914/1846559339.py:1: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['Date'] = pd.to_datetime(df['Date'], infer_datetime_format=True, format='mixed')


In [5]:
df.head()

,Date,Address,County,Eircode,Price (),Not Full Market Price,VAT Exclusive,Description of Property,Property Size Description
0,2010-01-01,"5 Braemor Drive, Churchtown, Co.Dublin",Dublin,NaN,"343,000.00",No,No,Second-Hand Dwelling house /Apartment,NaN
1,2010-03-01,"134 Ashewood Walk, Summerhill Lane, Portlaoise",Laois,NaN,"185,000.00",No,Yes,New Dwelling house /Apartment,greater than or equal to 38 sq metres and less...
2,2010-04-01,"1 Meadow Avenue, Dundrum, Dublin 14",Dublin,NaN,"438,500.00",No,No,Second-Hand Dwelling house /Apartment,NaN
3,2010-04-01,"1 The Haven, Mornington",Meath,NaN,"400,000.00",No,No,Second-Hand Dwelling house /Apartment,NaN
4,2010-04-01,"11 Melville Heights, Kilkenny",Kilkenny,NaN,"160,000.00",No,No,Second-Hand Dwelling house /Apartment,NaN


In [6]:
# Removing the Euro symbol from the Price column

def clean_price(string):
    string = string.replace(",","")
    return float(string[1:])
Price = df['Price ()'].apply(clean_price)

df = df.drop('Price ()',axis=1)

df['Price'] = Price

In [7]:
replacement_dict = {
    'Second-Hand Dwelling house /Apartment':'Second-Hand',
    'New Dwelling house /Apartment':'New',
    'Teach/Árasán Cónaithe Atháimhe':'Second-Hand',
    'Teach/Árasán Cónaithe Nua':'New',
    'Teach/?ras?n C?naithe Nua':'New'
}

df['Description of Property'] = df['Description of Property'].replace(replacement_dict)

In [8]:
df.head()

,Date,Address,County,Eircode,Not Full Market Price,VAT Exclusive,Description of Property,Property Size Description,Price
0,2010-01-01,"5 Braemor Drive, Churchtown, Co.Dublin",Dublin,NaN,No,No,Second-Hand,NaN,343000.0
1,2010-03-01,"134 Ashewood Walk, Summerhill Lane, Portlaoise",Laois,NaN,No,Yes,New,greater than or equal to 38 sq metres and less...,185000.0
2,2010-04-01,"1 Meadow Avenue, Dundrum, Dublin 14",Dublin,NaN,No,No,Second-Hand,NaN,438500.0
3,2010-04-01,"1 The Haven, Mornington",Meath,NaN,No,No,Second-Hand,NaN,400000.0
4,2010-04-01,"11 Melville Heights, Kilkenny",Kilkenny,NaN,No,No,Second-Hand,NaN,160000.0


In [9]:
Town = []
for i in range(len(df)):
    if df['Address'].loc[i].split(',')[-1][1:].split(" ")[-1].split('.')[-1].lower() == df['County'].loc[i].lower():
        if any(char.isdigit() for char in df['Address'].loc[i].split(',')[-2].strip()):
            string = re.sub(r"\d+","", df['Address'].loc[i].split(',')[-2].strip())
            Town.append(string.lower())
        else:
            Town.append(df['Address'].loc[i].split(',')[-2].strip().lower())
    else:
        Town.append(df['Address'].loc[i].split(',')[-1].strip().lower())

Town = pd.Series(Town)
Town = Town.str.capitalize()

In [10]:
df['Town'] = Town

In [11]:
df.head()

,Date,Address,County,Eircode,Not Full Market Price,VAT Exclusive,Description of Property,Property Size Description,Price,Town
0,2010-01-01,"5 Braemor Drive, Churchtown, Co.Dublin",Dublin,NaN,No,No,Second-Hand,NaN,343000.0,Churchtown
1,2010-03-01,"134 Ashewood Walk, Summerhill Lane, Portlaoise",Laois,NaN,No,Yes,New,greater than or equal to 38 sq metres and less...,185000.0,Portlaoise
2,2010-04-01,"1 Meadow Avenue, Dundrum, Dublin 14",Dublin,NaN,No,No,Second-Hand,NaN,438500.0,Dublin 14
3,2010-04-01,"1 The Haven, Mornington",Meath,NaN,No,No,Second-Hand,NaN,400000.0,Mornington
4,2010-04-01,"11 Melville Heights, Kilkenny",Kilkenny,NaN,No,No,Second-Hand,NaN,160000.0,melville heights


In [13]:
df['year'] = pd.DatetimeIndex(df['Date']).year
df['month'] = pd.DatetimeIndex(df['Date']).month

In [20]:
# Group the data by county and calculate the mean and median property prices
county_stats = df.groupby(['County','year'])['Price'].agg('median')

# Print the results
county_stats

County   year
Carlow   2010    157709.000
         2011    127000.000
         2012    100000.000
         2013     90000.000
         2014    110000.000
                    ...    
Wicklow  2020    315000.000
         2021    371022.000
         2022    383629.975
         2023    396475.770
         2024    400000.000
Name: Price, Length: 390, dtype: float64

In [23]:
county_stats.keys()

MultiIndex([( 'Carlow', 2010),
            ( 'Carlow', 2011),
            ( 'Carlow', 2012),
            ( 'Carlow', 2013),
            ( 'Carlow', 2014),
            ( 'Carlow', 2015),
            ( 'Carlow', 2016),
            ( 'Carlow', 2017),
            ( 'Carlow', 2018),
            ( 'Carlow', 2019),
            ...
            ('Wicklow', 2015),
            ('Wicklow', 2016),
            ('Wicklow', 2017),
            ('Wicklow', 2018),
            ('Wicklow', 2019),
            ('Wicklow', 2020),
            ('Wicklow', 2021),
            ('Wicklow', 2022),
            ('Wicklow', 2023),
            ('Wicklow', 2024)],
           names=['County', 'year'], length=390)

In [34]:
county_stats.get('Carlow').keys

<bound method Series.keys of year
2010    157709.00
2011    127000.00
2012    100000.00
2013     90000.00
2014    110000.00
2015    118000.00
2016    135000.00
2017    137000.00
2018    165000.00
2019    168067.79
2020    175000.00
2021    190000.00
2022    209500.00
2023    229294.27
2024    241455.50
Name: Price, dtype: float64>

In [16]:
county_stats[county_stats['County']=='Dublin']

KeyError: 'County'